# Lineout analysis

## Import data

**2021/22** (1st XV only) and **2022/23** (1st XV and 2nd XV)

Columns:
- `#` and `Half` (nth lineout per game and which half)
- `Date`	
- `Opposition`
- `Numbers` (how many in the line)	
- `Call`
- `Movement` (whether the move was static or dynamic)
- `Front`/`Middle`/`Back` (area of the lineout targeted)
- `Drive`, `Transfer` and `Flyby` (moves after the catch, if not off the top)
- `Hooker` and `Jumper`	(players involved)
- `Won` and	`Error` (successful or not, and reason if not)

In [22]:
import pandas as pd

pd.options.display.max_rows = 100
pd.options.display.max_columns = 30

df = pd.read_csv('EG Stats - 2022_23 - 1st XV Lineouts.csv', true_values=['x','Y'], false_values=['', 'N'], skipinitialspace=True, na_filter=False, skiprows=2).loc[:, "#":"Error"]
df["Team"] = "1st"
# df3 = pd.read_csv('EG Stats - 2022_23 - 2nd XV Lineouts.csv', true_values=['x','Y'], false_values=['', 'N'], skipinitialspace=True, na_filter=False, skiprows=2).loc[:, "#":"Error"]
# df3["Team"] = "2nd"
# df3["Season"] = "2022/23"

df

,#,Half,Season,Date,Opposition,Numbers,Call,Movement,Front,Middle,Back,Drive,Transfer,Flyby,Hooker,Jumper,Won,Error,Team
0,1,1,2021/22,2021-08-28,London Irish,5,,False,False,True,False,False,False,,Ben,Sam,True,,1st
1,2,1,2021/22,2021-08-28,London Irish,4,Snap,False,True,False,False,False,False,,Ben,Gus,True,,1st
2,3,1,2021/22,2021-08-28,London Irish,6,Even,False,False,True,False,True,False,,Ben,Sam,True,,1st
3,4,1,2021/22,2021-08-28,London Irish,6,,False,False,True,False,True,False,,Ben,Sam,True,,1st
4,5,1,2021/22,2021-08-28,London Irish,6,Green,False,False,False,True,False,False,,Ben,Rory,True,,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,9,2,2022/23,2023-04-29,Wensleydale,6,,False,False,True,False,False,False,2,Ben,Sam,True,,1st
282,10,2,2022/23,2023-04-29,Wensleydale,4,Yes,True,False,False,True,False,False,,Ben,Sam,True,,1st
283,11,2,2022/23,2023-04-29,Wensleydale,5,,False,False,True,False,False,False,,Ben,Sam,True,,1st
284,12,2,2022/23,2023-04-29,Wensleydale,6,RD,True,False,True,False,True,False,,Ben,Sam,True,Crusaders,1st


## Clean and derive columns

- `Position` - Front/Middle/Back
- `Play` - Off the top/Drive/Transfer/Flyby/Lost

Make SQL queryable with DuckDB

In [3]:
import duckdb

df.Hooker = ["Ben T" if f in ["Ben", "Totty"] else f for f in df["Hooker"]]
df.Flyby = [None if f not in ['1','2','3'] else f for f in df["Flyby"]]
df.Call = df.Call.replace('', '[wave]')
df['Result'] = ['Won' if r else 'Lost' for r in df.Won]
df['Position'] = ['Front' if d["Front"] else 'Middle' if d["Middle"] else 'Back' for d in df.to_dict("records")]
df['N'] = df['Numbers'].astype('str')+"-man"

def get_play(d):
    if d['Result'] == "Lost":
        return "Lost"
    elif d['Drive']:
        if d['Transfer']:
            return "Transfer"
        else:
            return "Drive"
    elif d['Flyby']:
        if d['Transfer']:
            return f"Transfer Flyby {d['Flyby']}"
        else:
            return f"Flyby {d['Flyby']}"
    else:
        return "Off the top"

df['Play'] = [get_play(d) for d in df.to_dict("records")]

df["Jumper"] = df["Jumper"].str.replace("John ", "John")

df.head(10)

con = duckdb.connect()
con.execute("CREATE TABLE df AS SELECT * FROM df")
con.query("SELECT * FROM df limit 10").df()

,#,Half,Season,Date,Opposition,Numbers,Call,Movement,Front,Middle,Back,Drive,Transfer,Flyby,Hooker,Jumper,Won,Error,Team,Result,Position,N,Play
0,1,1,2021/22,2021-08-28,London Irish,5,[wave],False,False,True,False,False,False,NaN,Ben T,Sam,True,,1st,Won,Middle,5-man,Off the top
1,2,1,2021/22,2021-08-28,London Irish,4,Snap,False,True,False,False,False,False,NaN,Ben T,Gus,True,,1st,Won,Front,4-man,Off the top
2,3,1,2021/22,2021-08-28,London Irish,6,Even,False,False,True,False,True,False,NaN,Ben T,Sam,True,,1st,Won,Middle,6-man,Drive
3,4,1,2021/22,2021-08-28,London Irish,6,[wave],False,False,True,False,True,False,NaN,Ben T,Sam,True,,1st,Won,Middle,6-man,Drive
4,5,1,2021/22,2021-08-28,London Irish,6,Green,False,False,False,True,False,False,NaN,Ben T,Rory,True,,1st,Won,Back,6-man,Off the top
5,6,1,2021/22,2021-08-28,London Irish,5,[wave],False,False,True,False,False,False,NaN,Ben T,Sam,False,Throw,1st,Lost,Middle,5-man,Lost
6,7,2,2021/22,2021-08-28,London Irish,4,Yes,True,False,False,True,False,False,NaN,Ben T,Sam,False,Throw,1st,Lost,Back,4-man,Lost
7,8,2,2021/22,2021-08-28,London Irish,4,Snap,False,True,False,False,False,False,NaN,Ben T,Gus,True,,1st,Won,Front,4-man,Off the top
8,9,2,2021/22,2021-08-28,London Irish,4,Snap,False,True,False,False,False,False,NaN,Ben T,Gus,False,Stolen,1st,Lost,Front,4-man,Lost
9,10,2,2021/22,2021-08-28,London Irish,5,RD,True,False,True,False,False,False,NaN,Ben T,Sam,True,,1st,Won,Middle,5-man,Off the top


## Charts

In [4]:
import altair as alt

# Color scales
position_scale = alt.Scale(
    domain=['Front', 'Middle', 'Back'], 
    range=['red', 'orange', 'green']
)

n_scale = alt.Scale(
    domain=[4,5,6,7], 
    range=["#ca0020", "#f4a582", "#92c5de", "#0571b0"]
)

call_scale = alt.Scale(
    domain=['Matlow', 'Red', 'Orange', 'Plus', 'RD', 'Even', 'Odd', 'Green', 'Yes', 'No', 'Snap', '[wave]'], 
    range=['black', 'red', 'red', 'dark_orange', 'dark_orange', 'orange', 'orange', 'green', 'green', 'red', 'red', 'orange']
)

play_scale = alt.Scale(
    domain=['Off the top', 'Flyby 1', 'Flyby 2', 'Flyby 3', 'Drive', 'Transfer', 'Lost'], 
    range=['green', 'orange', 'orange', 'orange', 'red', 'red', 'black']
)

# Sort orders
position_order = ["Front", "Middle", "Back"]
call_order = ['Matlow','Snap','Red','Orange','Plus','No','RD','[wave]','Even','Odd','Yes','Green']

## Chart defs

#### Count by Area/Call/Play
`count_by(df, by=["Area", "Call", "Play"])`

In [33]:
def count_by(df, by=["Area", "Call", "Play"]):
    if by=="Area":
        col = "Position"
        sort = position_order
        scale = position_scale
        title="Most Used Target Area"
    elif by=="Call":
        col = "Call"
        sort = call_order
        scale = call_scale
        title="Most Used Calls"
    elif by=="Play":
        col = "Play"
        sort="-x"
        scale = play_scale
        title="Most Used Plays from Lineout"

    chart = alt.Chart(df).mark_bar().encode(
        x = alt.X("count()", title="Number of lineouts"),
        y = alt.Y(f"{col}:O", title='', sort=sort),
        color=alt.Color(f'{col}:O', scale=scale, legend=None),
        opacity=alt.Opacity("count():Q",legend=None),
        row=alt.Row("N:O",title="", spacing=10),
        tooltip = [
            col,
            alt.Tooltip("count()", title="Count"), 
            alt.Tooltip("sum(pct):Q", title="% of total", format=".0%"), 
            alt.Tooltip("mean(Won):Q", format=".0%", title="Success rate")
        ]
    ).resolve_scale(
        y='independent',
        opacity='independent'
    ).properties(title=title)

    if by == "Play":
        chart = chart.transform_filter(alt.datum.Play != 'Lost')

    return chart

### Success by Area/Call
`success_bar(df, by=["Area", "Call"])`

In [6]:
def success_bar(df, by=["Area", "Call"]):
    
    if by=="Area":
        col = "Position"
        sort = position_order
        scale = position_scale
        title = "Most Successful Target Area"
    elif by=="Call":
        col = "Call"
        sort = call_order
        scale = call_scale
        title = "Most Successful Calls"

    chart = alt.Chart(df).mark_bar().encode(
        y = alt.Y(f'{col}:O', title='', sort=sort),
        x = alt.X("mean(Won):Q", axis=alt.Axis(title = "Success rate", format=".0%")),
        color = alt.Color(f'{col}:O', scale=scale, legend=None),
        opacity = alt.Opacity("mean(Won):Q",legend=None),
        row = alt.Row("N",title="", spacing=10),
        tooltip = [
            col, 
            alt.Tooltip("count()", title="Count"), 
            alt.Tooltip("sum(pct):Q", title="% of total", format=".0%"), 
            alt.Tooltip("mean(Won):Q", format=".0%", title="Success rate")
        ]
    ).resolve_scale(
        y ='independent',
    ).properties(title=title)

    return chart


### Success rate + histogram
`success_hist(df, by=["Numbers", "Area", "Play"])`

In [7]:
def success_hist(df, by=["Numbers", "Area", "Play"]):
    col1 = by
    col2 = by
    subtitle = ""
    opacity = alt.value(0.5)

    if by=="Numbers":
        col1 = "N"
        title = "Lineout numbers"
        x_title = "Numbers"
        scale = n_scale
        order = "Numbers:N",
        sort = "x"
    elif by=="Area":
        col1 = "Position"
        col2 = "Position"
        title = "Target area"
        x_title = "Target"
        scale = position_scale
        order = "position_order:N"
        sort = position_order
    elif by=="Play":
        title = "Most common outcomes"
        subtitle = "How the ball is played after the throw"
        x_title = "Plays off lineout"
        scale = play_scale
        sort = "-y"
        order = []
        opacity = alt.condition(alt.datum.Play=="Lost", alt.value(1), alt.value(0.7))

    base = alt.Chart(df).encode(
        x = alt.X(f'{col1}:N', title=x_title, axis=alt.Axis(labelAngle=0), sort=sort),
    )
    success = base.mark_line(point=True, color="black").encode(
        y=alt.Y("mean(Won):Q", axis=alt.Axis(title = "Success rate", format=".0%"), sort=None, scale=alt.Scale(domain=[0,1])),
        order=order
    ).transform_calculate(
        position_order="{'Front':0, 'Middle':1, 'Back':2}[datum.Position]"
    )
    success_text = success.mark_text(dy=15, size=14).encode(
        text = alt.Text("mean(Won):Q", format=".0%")
    )
    bar = base.mark_bar().encode(#
        y=alt.Y("count():Q", sort=None, title=title),
        color=alt.Color(f'{col2}', scale=scale, title="", legend=None),
        order=order,
        opacity=opacity
    ).transform_calculate(
        position_order="{'Front':0, 'Middle':1, 'Back':2}[datum.Position]"
    )
    bar_text = bar.mark_text(dy=-10, size=14).encode(
        text = alt.Text("count():Q"),
    )

    if by!="Play":
        chart = alt.layer(
            bar + bar_text, success + success_text
            ).resolve_scale(
                color='independent', 
                y="independent",
            )
    else:
        chart = (bar + bar_text)

    chart = chart.properties(
            width=300,
            title=alt.TitleParams(text=title, subtitle=subtitle, anchor="start"),
    )

    return chart


### Hooker/Jumper success rate
`player_success(df, pos=["Hooker", "Jumper"], min=5)`

In [8]:
def player_success(df, pos=["Hooker", "Jumper"], min=5):
    

    base = alt.Chart(df).transform_aggregate(
        cnt='count()',
        success="mean(Won)",
        groupby=[pos]
    ).transform_filter(
        f"datum.cnt >= {min}"
    ).encode(
        x = alt.X(f'{pos}:O', title=pos, axis=alt.Axis(labelAngle=0), sort="y"),
        y = alt.Y("success:Q", axis=alt.Axis(title="Success rate", format=".0%"), scale=alt.Scale(domain=[0,1])),
        order="success:Q"
    )

    bar = base.mark_bar().encode(
        color=alt.Color("cnt:Q", scale=alt.Scale(scheme='blues'), legend=None)
    )

    success_text = base.mark_text(dy=-10, size=14).encode(
        text=alt.Text("cnt:Q")
    )

    chart = alt.layer(
        bar,
        success_text
    ).resolve_scale(
        color="independent"
    ).properties(
        # width=alt.Step(30), 
        width=300, 
        title=alt.TitleParams(
            text=f"{pos}s used", 
            subtitle=f"with a minimum of {min} lineouts taken", 
            subtitleFontSize=10, 
            anchor="start")
    )

    return chart


### Heatmaps

`heatmap(df, by=["Area", "Call", "Play"])`

In [9]:
def heatmap(df, by=["Area", "Call", "Play"]):
    if by=="Area":
        col = "Position"
        sort = position_order
        scale = position_scale
        sub="target area"
    elif by=="Call":
        col = "Call"
        sort = call_order
        scale = call_scale
        sub="call"
    elif by=="Play":
        col = "Play"
        sort="-x"
        scale = play_scale
        sub="play from lineout"
    
    base = alt.Chart(df).encode(
        x=alt.X('N:O', title='', axis=alt.Axis(labelAngle=0)),
        y=alt.Y(f'{col}:O', title='', sort=sort),
    ).properties(width=250, height=alt.Step(30))

    success = base.mark_rect().encode(
        color=alt.Color('mean(Won):Q', scale=alt.Scale(scheme='blues', domain=[0.75,1]), legend=None),
    ).properties(title=alt.TitleParams("Most Successful Options", subtitle=f"Success rate by {sub}"))

    success_text = base.mark_text(fontWeight='bold', fontSize=16).encode(
        text=alt.Text('mean(Won):Q', format='.0%'),
        color=alt.Color('mean(Won):Q', scale=alt.Scale(scheme='greys', reverse=True), legend=None)
    )

    count = base.mark_rect().encode(
        color=alt.Color('count()', scale=alt.Scale(scheme='oranges'), legend=None)
    ).properties(title=alt.TitleParams("Most Used Options", subtitle=f"Number of lineouts by {sub}", subtitleFontSize=10, anchor='start'))

    count_text = base.mark_text(fontWeight='bold', fontSize=18).encode(
        text=alt.Text('count():Q'),
        color=alt.Color('count()', scale=alt.Scale(scheme='greys', reverse=True), legend=None)
    )

    chart = (count + count_text).resolve_scale(color='independent') 

    if by!="Play":
        chart = chart | (success + success_text).resolve_scale(color='independent')
    return chart

## Display

In [34]:
# heatmap(df, "Area") & heatmap(df, "Call") & heatmap(df, "Play")

# # Area / Call
# success_bar(df, "Call")

# Numbers / Area / Play
# success_hist(df, "Play")

# player_success(df, "Jumper")

count_by(df, "Area")

ValueError: Position encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

In [26]:
# data = df[df['Opposition']=="Burgess Hill"]
col = "Call"
sort = call_order
scale = call_scale
title = "Most Successful Calls"

chart = alt.Chart(df).mark_point().encode(
    x = alt.X('count():Q', title=''),
    y = alt.Y("mean(Won):Q", axis=alt.Axis(title = "Success rate", format=".0%")),
    color = alt.Color(f'{col}:O', scale=scale, legend=None),
    shape = "Numbers:O",
    size="count():Q",
    tooltip = [
        col, 
        alt.Tooltip("count()", title="Count"), 
        alt.Tooltip("sum(pct):Q", title="% of total", format=".0%"), 
        alt.Tooltip("mean(Won):Q", format=".0%", title="Success rate")
    ]
).properties(title=title)

chart

alt.Chart(...)

### Outputs

In [13]:
data = con.query("SELECT * FROM df WHERE team=='1st'").df()

# count_by(data, "Call")
#success_bar(df, "Call")
# success_hist(data, "Play")

In [15]:
from altair_saver import save

save(count_area_bar, "target_area_count.png")
save(success_area_bar, "target_area_success.png")
save(count_call_bar, "calls_count.png")
save(success_call_bar, "calls_success.png")
save(count_play_bar, "outcome_count.png")
save(jumpers, "jumpers.png")
save(target, "target_area.png")
save(numbers, "numbers.png")
save(plays, "outcome.png")
save(heatmaps, "heatmaps.png")

NameError: name 'count_area_bar' is not defined

In [ ]:
alt.renderers.enable('altair_saver', fmts=['vega-lite', 'png'])

RendererRegistry.enable('altair_saver')